<a href="https://colab.research.google.com/github/fabiollg/PI-4_Predicao_Valores_Imoveis_SJC/blob/main/PI_4_Predicao_Valores_Imoveis_SJC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Carregando dados do arquivo "price_versions.zip"

In [1]:
import pandas as pd
  
url_dados = 'https://github.com/fabiollg/PI-4_Predicao_Valores_Imoveis_SJC/blob/main/dados/new-price_versions.zip?raw=true'
dados = pd.read_csv(url_dados, compression = 'zip')
dados

,id,listing_id,price,price_monthly_condo_fee,price_rental_total,price_yearly_iptu,scraped_at
0,332687,2513233975,1400,471,0,52,2021-04-23 05:00:01.015793+00
1,332727,2517326136,2800,320,0,57,2021-04-23 05:00:17.721471+00
2,332767,2516892648,1900,635,0,0,2021-04-23 05:00:34.659799+00
3,332807,2517463901,1300,390,0,0,2021-04-23 05:00:50.954843+00
4,332847,2517054725,900,403,0,0,2021-04-23 05:01:06.874916+00
...,...,...,...,...,...,...,...
2745975,336766,2495377212,16000,2000,0,0,2021-04-24 05:09:12.992558+00
2745976,336806,2490859640,1800,730,0,0,2021-04-24 05:09:28.764087+00
2745977,336846,2517588087,3000,350,0,500,2021-04-24 05:09:54.776579+00
2745978,336886,2516740767,5200,690,0,0,2021-04-24 05:10:10.501659+00


## Carregando dados do arquivo "listings.zip"

In [2]:
url_dados2 = 'https://github.com/fabiollg/PI-4_Predicao_Valores_Imoveis_SJC/blob/main/dados/new-listings.zip?raw=true'
dados2 = pd.read_csv(url_dados2,compression='zip')
dados2

<ipython-input-2-38832b203b67>:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  dados2 = pd.read_csv(url_dados2,compression='zip')


,id,resource_id,provider_id,advertiser_id,address_country,address_state,address_city,address_neighborhood,address_street_number,address_street,...,price_monthly_condo_fee,price_rental_total,price_yearly_iptu,publication_type,listing_status,listing_type,created_at,updated_at,raw_data,scraped_at
0,923833,2539419421,61529.0,8132d7f5-ac6e-8229-ac29-66e53d55fe1e,BR,São Paulo,São José dos Campos,Jardim das Indústrias,NaN,NaN,...,482,0,155,STANDARD,NaN,USED,2021-10-28 01:43:54.123+00,2021-10-28 01:43:56.883+00,"{""link"": {""rel"": """", ""data"": {""city"": ""São Jos...",2021-10-28 05:07:15.43542+00
1,1336880,2549894537,10048.0,5edc5203-b48b-1278-0e4b-36b4bf556daf,BR,São Paulo,São José dos Campos,Jardim América,NaN,Rua Andorra,...,290,0,0,STANDARD,NaN,USED,2022-02-01 19:01:11.276+00,2022-02-01 19:01:21.341+00,"{""link"": {""rel"": """", ""data"": {""city"": ""São Jos...",2022-02-02 05:08:59.620765+00
2,1134517,2546641936,60384.0,ef34904e-bd8d-66bd-cd90-e3f966a6bf3b,BR,São Paulo,São José dos Campos,Jardim São Dimas,NaN,Avenida Doutor Adhemar de Barros,...,750,0,0,STANDARD,NaN,USED,2022-01-04 07:22:09.019+00,2022-01-28 11:21:26.848+00,"{""link"": {""rel"": """", ""data"": {""city"": ""São Jos...",2022-01-05 05:07:37.423878+00
3,1107421,2546130088,NaN,5c45578a-2454-7af7-162e-b80754623bbb,BR,São Paulo,São José dos Campos,Jardim Satélite,NaN,Rua Polar,...,238,0,0,STANDARD,NaN,USED,2021-12-27 20:52:57.74+00,2022-01-10 13:07:26.754+00,"{""link"": {""rel"": """", ""data"": {""city"": ""São Jos...",2021-12-28 05:10:28.48205+00
4,1536013,2557422239,46200.0,1404e2f2-c083-a256-152f-81c087dacf2e,BR,São Paulo,São José dos Campos,Jardim Augusta,NaN,NaN,...,390,0,59,STANDARD,NaN,USED,2022-04-06 01:16:39.038+00,2022-04-06 01:19:49.16+00,"{""link"": {""rel"": """", ""data"": {""city"": ""São Jos...",2022-04-06 05:06:23.146788+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60964,1072808,2545119128,63537.0,fae8f98b-2c16-bdef-d2b2-e611a3718473,BR,São Paulo,São José dos Campos,Jardim Oswaldo Cruz,22,Rua Siria,...,0,0,0,STANDARD,NaN,USED,2021-12-18 01:41:44.46+00,2021-12-24 01:52:36.69+00,"{""link"": {""rel"": """", ""data"": {""city"": ""São Jos...",2021-12-18 05:20:19.721142+00
60965,1039824,2543951968,56902.0,8e0b76b5-76ce-cdc7-c9ea-74b9347f68cc,BR,São Paulo,São José dos Campos,Vila Tesouro,NaN,NaN,...,250,0,35,STANDARD,NaN,USED,2021-12-08 14:27:38.954+00,2021-12-08 14:27:40.143+00,"{""link"": {""rel"": """", ""data"": {""city"": ""São Jos...",2021-12-09 05:02:21.780106+00
60966,976422,2541504617,46200.0,1404e2f2-c083-a256-152f-81c087dacf2e,BR,São Paulo,São José dos Campos,Jardim Esplanada II,NaN,NaN,...,1050,0,242,STANDARD,NaN,USED,2021-11-17 00:50:43.138+00,2022-03-30 01:06:23.812+00,"{""link"": {""rel"": """", ""data"": {""city"": ""São Jos...",2021-11-17 05:03:38.813772+00
60967,1015032,2542940581,51630.0,72cd5fe0-8473-589b-e305-8aff89ad70f9,BR,São Paulo,São José dos Campos,Jardim Alvorada,101,Avenida Cassiano Ricardo,...,811,0,1800,STANDARD,NaN,USED,2021-11-30 06:16:54.109+00,2021-12-08 10:55:32.824+00,"{""link"": {""rel"": """", ""data"": {""city"": ""São Jos...",2021-12-01 05:05:35.393929+00
